In [ ]:
#  Import packages

# Numpy,Pandas
import numpy as np
import pandas as pd
import datetime

# matplotlib,seaborn,pyecharts
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.pyplot import figure
import plotly.graph_objects as go
# plt.style.use('ggplot')  
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline
import missingno as msno

# ignore warnings
import warnings
warnings.filterwarnings('ignore')  
 
pd.set_option('display.float_format', lambda x: '%.4f' % x)

#importing the models to use 
from sklearn.ensemble import IsolationForest
# import / install deep svdd , see https://pypi.org/project/deep-svdd/ for more details 
# !pip install deep-svdd ##execute this just one time 
#from dsvdd import *
import keras
import tensorflow as tf
# importing metrics
from sklearn.metrics import recall_score, precision_score, f1_score

In [ ]:
# prepare data 
#mounting the driver so we can access the content of google drive
from google.colab import drive
drive.mount('/content/drive')

# reading the dataset
path = '/content/drive/MyDrive/'

train_data = pd.read_csv(path + 'Copie de train_dataset.csv')
valid_data = pd.read_csv(path + 'Copie de valid_dataset.csv')
test_data = pd.read_csv(path + 'Copie de test_dataset.csv')


train_fraud = train_data.pop('fraud')
test_fraud = test_data.pop('fraud')
valid_fraud  = valid_data.pop('fraud')

try:
  data = pd.read_pickle(path + 'fraud_dataset.pkl')
except:
  data = pd.read_csv(path + 'fraud_dataset.csv')
  # peakle data 
  data.to_pickle('/content/drive/MyDrive/fraud_dataset.pkl')


data['hour'] = pd.to_datetime(data['datetime']).dt.hour
data.drop('datetime', axis = 1, inplace = True)
# dropping the useless column 'Unnamed: 0'
data.drop('Unnamed: 0', axis = 1, inplace = True)

train_columns = set(['acquirerid', 'agriculture_list', 'rollingsum_card_merchant_24h','bankid', 'batchamount','card','emv','cup','construction_list','mean_merchant_amount','mcc', 'list_bankid_ci', 'list_bankid_b', 'fuel_list', 'pospayenvcode','merchant', 'medic_list',
                   'rollingcount_card_merchant_600s','riskmerchant', 'resp_code', 'rollingcount_merchant_24h','rollingcount_merchant_900s','rollingsum_card_24h','rollingsum_merchant_24h','rollingsum_merchant_900s',
                   'hour', 'trc', 'travel_list', 'terminal', 'super_market_list'])

train_columns = set(train_columns) & set(train_data.columns) & set(test_data.columns) & set(valid_data.columns)
train_columns = list(train_columns)

# let's shuffle the data first
data = data.sample(frac=1).reset_index(drop = True)

data  = data.iloc[:200000]
train_data_ = data[:150000]
train_data_ = train_data_[train_data_.fraud == 0]
X_train = train_data_[train_columns]

# test_data = data[150000:]
# X_test,Y_test = test_data[train_columns], test_data['fraud']


Mounted at /content/drive


In [ ]:
train_data, test_data, valid_data = train_data[train_columns], test_data[train_columns], valid_data[train_columns]


# Normalizing data 
from sklearn.preprocessing import StandardScaler as sc


# #train_data.drop(['Unnamed: 0', 'cup'], axis = 1, inplace = True)
# #test_data.drop(['Unnamed: 0', 'cup'], axis = 1, inplace = True)
# valid_data.drop(['Unnamed: 0', 'cup'], axis = 1, inplace = True)

# model = sc()

X_train = X_train.values
# X_train = model.fit_transform(X = X_train)

train_data = train_data.values
# train_data = model.transform(X = train_data)

test_data = test_data.values
# test_data = model.transform(X = test_data)

valid_data = valid_data.values
# valid_data = model.transform(X = valid_data)

In [ ]:
# I need to define the architecture of the network
def network_model(input_shape, H=10):

  model = keras.models.Sequential()

  model.add(keras.layers.Dense(units= input_shape ,activation='relu', use_bias=False))
  model.add(keras.layers.Dropout(0.2))
  model.add(keras.layers.Dense(units= 16, activation='relu', use_bias=False))
  model.add(keras.layers.Dropout(0.2))
  model.add(keras.layers.Dense(units= 8, activation='relu', use_bias=False))
  model.add(keras.layers.Dropout(0.2))
  model.add(keras.layers.Dense(units= 4, activation='relu', use_bias=False))
  model.add(keras.layers.Dense(units= H, activation='relu', use_bias=False))
  
  return model

In [ ]:
from contextlib import contextmanager
import matplotlib.pyplot as plt
import numpy as np


__all__ = ['plot_most_normal_and_abnormal_images', 'task']


@contextmanager
def task(_=''):
    yield

In [ ]:
import tensorflow as tf
import numpy as np
from math import ceil
from sklearn.metrics import roc_auc_score
from tensorflow import keras
from tqdm import tqdm

#from .utils import task
tf.compat.v1.disable_eager_execution()


class DeepSVDD:
  def __init__(self, keras_model, input_shape=(28, 28, 1), objective='one-class',
                nu=0.1, representation_dim=32, batch_size=128, lr=1e-3):
      self.represetation_dim = representation_dim
      self.objective = objective
      self.keras_model = keras_model
      self.nu = nu
      self.R = tf.compat.v1.get_variable('R', [], dtype=tf.float32, trainable=False)
      self.c = tf.compat.v1.get_variable('c', [self.represetation_dim], dtype=tf.float32, trainable=False)
      self.warm_up_n_epochs = 10
      self.batch_size = batch_size

      with task('Build graph'):
          self.x = tf.compat.v1.placeholder(tf.float32, [None] + list(input_shape))
          self.latent_op = self.keras_model(self.x)
          self.dist_op = tf.reduce_sum(tf.square(self.latent_op - self.c), axis=-1)

          if self.objective == 'soft-boundary':
              self.score_op = self.dist_op - self.R ** 2
              penalty = tf.maximum(self.score_op, tf.zeros_like(self.score_op))
              self.loss_op = self.R ** 2 + (1 / self.nu) * penalty

          else:  # one-class
              self.score_op = self.dist_op
              self.loss_op = self.score_op

          opt = tf.compat.v1.train.AdamOptimizer(lr)
          self.train_op = opt.minimize(self.loss_op)

      config = tf.compat.v1.ConfigProto()
      config.gpu_options.allow_growth = True
      self.sess = tf.compat.v1.Session(config=config)
      self.sess.run(tf.compat.v1.global_variables_initializer())

  def __del__(self):
      self.sess.close()

  def fit(self, X, X_test, y_test, epochs=10, verbose=True):
      N = X.shape[0]
      BS = self.batch_size
      BN = int(ceil(N / BS))

      self.sess.run(tf.compat.v1.global_variables_initializer())
      self._init_c(X)

      ops = {
          'train': self.train_op,
          'loss': tf.reduce_mean(self.loss_op),
          'dist': self.dist_op
      }
      keras.backend.set_learning_phase(True)

      for i_epoch in range(epochs):
          ind = np.random.permutation(N)
          x_train = X[ind]
          g_batch = tqdm(range(BN)) if verbose else range(BN)
          for i_batch in g_batch:
              x_batch = x_train[i_batch * BS: (i_batch + 1) * BS]
              results = self.sess.run(ops, feed_dict={self.x: x_batch})

              if self.objective == 'soft-boundary' and i_epoch >= self.warm_up_n_epochs:
                  self.sess.run(tf.compat.v1.assign(self.R, self._get_R(results['dist'], self.nu)))

          else:
              if verbose:
                  pred = self.predict(X_test)  # pred: large->fail small->pass
                  auc = roc_auc_score(y_test, -pred)  # y_test: 1->pass 0->fail
                  print('\rEpoch: %3d AUROC: %.3f' % (i_epoch, auc))

  def predict(self, X):
      N = X.shape[0]
      BS = self.batch_size
      BN = int(ceil(N / BS))
      scores = list()
      keras.backend.set_learning_phase(False)

      for i_batch in range(BN):
          x_batch = X[i_batch * BS: (i_batch + 1) * BS]
          s_batch = self.sess.run(self.score_op, feed_dict={self.x: x_batch})
          scores.append(s_batch)

      return np.concatenate(scores)

  def _init_c(self, X, eps=1e-1):
      N = X.shape[0]
      BS = self.batch_size
      BN = int(ceil(N / BS))
      keras.backend.set_learning_phase(False)

      with task('1. Get output'):
          latent_sum = np.zeros(self.latent_op.shape[-1])
          for i_batch in range(BN):
              x_batch = X[i_batch * BS: (i_batch + 1) * BS]
              latent_v = self.sess.run(self.latent_op, feed_dict={self.x: x_batch})
              latent_sum += latent_v.sum(axis=0)

          c = latent_sum / N

      with task('2. Modify eps'):
          c[(abs(c) < eps) & (c < 0)] = -eps
          c[(abs(c) < eps) & (c > 0)] = eps

      self.sess.run(tf.compat.v1.assign(self.c, c))

  def _get_R(self, dist, nu):
      return np.quantile(np.sqrt(dist), 1 - nu)


In [ ]:
# Now let's build our model

input_shape = X_train.shape[1]
tf.compat.v1.reset_default_graph()
# build model and DeepSVDD
keras_model = network_model(input_shape, 4)
svdd = DeepSVDD(keras_model, input_shape= (input_shape,), representation_dim=4,
                objective='one-class')


# train DeepSVDD
svdd.fit(X_train, valid_data, valid_fraud, epochs=10, verbose=False)

# test DeepSVDD

In [ ]:
# score = svdd.predict(X_test)
# # Now let's find the best threeshold 
# threeshold = np.quantile(svdd.predict(X_train),0.95) #setting threeshold
# predicted = np.where(score>threeshold, 1,0)

In [ ]:
from collections import Counter
Counter(test_fraud)

Counter({0: 24980, 1: 20})

In [ ]:
Counter(valid_fraud)

Counter({0: 24974, 1: 26})

In [ ]:
score = svdd.predict(valid_data)
y_true = valid_fraud
threeshold = np.quantile(svdd.predict(train_data),0.95) #setting threeshold
predicted = np.where(score>threeshold, 1,0)

print(f'the recall is :{recall_score(y_true,predicted)}', end = '\n')
print(f'the precision is :{precision_score(y_true,predicted)}', end = '\n')
print(f'the f1 score is :{f1_score(y_true,predicted)}', end = '\n')

the recall is :0.4230769230769231
the precision is :0.008258258258258258
the f1 score is :0.016200294550810013


In [ ]:
score = svdd.predict(valid_data)
y_true = test_fraud
# Now let's find the best threeshold 
threeshold_quantiles = np.linspace(0,1,20)
recall_scores = []
precision_scores = []
f1_scores = []
for threeshold_quantile in threeshold_quantiles:
  threeshold = np.quantile(svdd.predict(train_data),threeshold_quantile) #setting threeshold
  predicted = np.where(score>threeshold, 1,0)
  recall_scores.append(recall_score(y_true,predicted))
  precision_scores.append(precision_score(y_true,predicted))
  f1_scores.append(f1_score(y_true,predicted))

# plotting the figure 

In [ ]:
import plotly.express as px

fig = go.Figure()
# adding traces 
names = ['recall', 'precision', 'f1_scores']
for i,y in enumerate([recall_scores, precision_scores, f1_scores]):
  fig.add_trace(go.Scatter(
      x = threeshold_quantiles,
      y = y,
      name = names[i]
  ))
fig.show()

In [ ]:
svdd.predict(valid_data)

array([130705.734,  21263.645, 189284.28 , ..., 652565.   ,  90123.24 ,
        22952.852], dtype=float32)

In [ ]:
score

array([130705.734,  21263.645, 189284.28 , ..., 652565.   ,  90123.24 ,
        22952.852], dtype=float32)

In [ ]:
# predicting on test and valid_data 
# for valid_data 
score = pd.DataFrame({'valid_data' : svdd.predict(valid_data),
                      'test_data' : svdd.predict(test_data)})
score = pd.concat([score, pd.DataFrame({'train_data': svdd.predict(train_data)})], axis = 1)
score.to_csv("drive/MyDrive/results_SVDD.csv")


In [ ]:
# import plotly.express as px

# fig = go.Figure()
# # adding traces 
# names = ['recall', 'precision', 'f1_scores']
# for i,y in enumerate([recall_scores, precision_scores, f1_scores]):
#   fig.add_trace(go.Scatter(
#       x = threeshold_quantiles,
#       y = y,
#       name = names[i]
#   ))
# fig.show()

In [ ]:
best_threes = np.quantile(svdd.predict(train_data),0.947)
predicted = np.where(score>best_threes, 1,0)
y_true = test_fraud
print(f'the recall is :{recall_score(y_true,predicted)}', end = '\n')
print(f'the precision is :{precision_score(y_true,predicted)}', end = '\n')
print(f'the f1 score is :{f1_score(y_true,predicted)}', end = '\n')

In [ ]:
estimators_number = np.arange(1,500,step = 50)
recall_scores = []
precision_scores = []
f1_scores = []
y_true = Y_test

for n_estimators in estimators_number:
  model_if = IsolationForest(n_estimators = n_estimators, random_state=2022, max_features= 20).fit(X_train)
  predicted = model_if.predict(X_test)
  predicted = np.where(predicted == 1,0,1)
  recall_scores.append(recall_score(y_true,predicted))
  precision_scores.append(precision_score(y_true,predicted))
  f1_scores.append(f1_score(y_true,predicted))


In [ ]:
fig = go.Figure()
# adding traces 
names = ['recall', 'precision', 'f1_scores']
for i,y in enumerate([recall_scores, precision_scores, f1_scores]):
  fig.add_trace(go.Scatter(
      x = estimators_number,
      y = y,
      name = names[i]
  ))
fig.show()

In [ ]:
# Pourquoi ça stagne ????

In [ ]:
# Let's try to use some bayesian optimization:


# Bayesien optimization:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# reading the dataset
path = '/content/drive/MyDrive/'
try:
  data = pd.read_pickle(path + 'fraud_dataset.pkl')
except:
  data = pd.read_csv(path + 'fraud_dataset.csv')
  # peakle data 
  data.to_pickle('/content/drive/MyDrive/fraud_dataset.pkl')


data['hour'] = pd.to_datetime(data['datetime']).dt.hour
data.drop('datetime', axis = 1, inplace = True)
# dropping the useless column 'Unnamed: 0'
data.drop('Unnamed: 0', axis = 1, inplace = True)

train_columns = ['acquirerid', 'agriculture_list', 'rollingsum_card_merchant_24h','bankid', 'batchamount','card','emv','cup','construction_list','mean_merchant_amount','mcc', 'list_bankid_ci', 'list_bankid_b', 'fuel_list', 'pospayenvcode','merchant', 'medic_list',
                   'rollingcount_card_merchant_600s','riskmerchant', 'resp_code', 'rollingcount_merchant_24h','rollingcount_merchant_900s','rollingsum_card_24h','rollingsum_merchant_24h','rollingsum_merchant_900s',
                   'hour', 'trc', 'travel_list', 'terminal', 'super_market_list']

# let's shuffle the data first
data = data.sample(frac=1).reset_index(drop = True)

data  = data.iloc[:1000000]
frauds = data.pop('fraud')

In [ ]:
#data, frauds = data.values, frauds.values
# define search space
from sklearn.model_selection import RepeatedStratifiedKFold
try:
  from skopt import BayesSearchCV
except:
  !pip install scikit-optimize
  from skopt import BayesSearchCV

params = dict()
params['n_estimators'] = (1, 1000, 'log-uniform')
params['contamination'] = (0.01, 0.12, 'uniform')
params['max_features'] = (1,30)
params['bootstrap'] = [True, False]

model_if = IsolationForest()

# define evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

def custom_f1_score(estimator,X,y):
  return f1_score(y, np.where(estimator.predict(X) == 1,0,1))
  
# define the search
search = BayesSearchCV(estimator= model_if , search_spaces=params, n_jobs=-1, cv=cv, scoring = custom_f1_score)



In [ ]:
X = data[train_columns].values
Y = frauds.values

search.fit(X,frauds)

In [ ]:
# Now let's implement Isolation forest 


model_if = IsolationForest(n_estimators = 100, random_state=2022).fit(train_data)

predicted = pd.DataFrame({
    'valid_data': model_if.predict(valid_data),
    'test_data': model_if.predict(test_data)
})

predicted = pd.concat([score, pd.DataFrame({'train_data': model_if.predict(train_data)})], axis = 1)

predicted.to_csv("drive/MyDrive/results_IsolationForest.csv")
# predicted = model_if.predict(X_test)
# predicted = np.where(predicted == 1,0,1)

# y_true = Y_test
# print(f'the recall is :{recall_score(y_true,predicted)}', end = '\n')
# print(f'the precision is :{precision_score(y_true,predicted)}', end = '\n')
# print(f'the f1 score is :{f1_score(y_true,predicted)}', end = '\n')

In [ ]:
sasas

In [ ]:
predicted = model_if.predict(test_data)
predicted = np.where(predicted == 1,0,1)

y_true = test_fraud
print(f'the recall is :{recall_score(y_true,predicted)}', end = '\n')
print(f'the precision is :{precision_score(y_true,predicted)}', end = '\n')
print(f'the f1 score is :{f1_score(y_true,predicted)}', end = '\n')